# Model 1 - MCMC with Turing.jl

## Install Turing.jl (only need to do this once)
See the Turing instructions on their [wiki](https://github.com/yebai/Turing.jl/wiki/Get-started#install-turingjl) for more info.

In [1]:
#Pkg.add("Turing")
#Pkg.update()
#Pkg.checkout("Turing", "master") 
#Pkg.build("Turing")
#Pkg.test("Turing")

## Setup

In [2]:
using Turing

[Turing]: AD chunk size is set as 40


In [3]:
using Distributions, StatsFuns
using Mamba: describe, plot

## STEP 1: define data

In [4]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

## STEP 2: conduct inference
First we are going to define a model. This is the way of doing it for the Turing package. I managed to convert this over from the JAGS specification (see the original Matlab repo) with not so much trouble. It's pretty nice in fact.

In [5]:
@model model1(k, Δμ, T) = begin
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    σ² ~ Uniform(0,100)
    for c ∈ 1:length(k)
        k[c] ~ Binomial(T, PC(Δμ[c],σ²))
    end
    return σ²
end

model1 (generic function with 4 methods)

Now do sampling. Note that there are a variety of sampling algorithms we can use.

In [6]:
n_samples = 300
samples = sample( model1(data["k"], data["Δμ"], data["T"]), PG(50,n_samples))
σ²posterior = samples[:σ²]

# TODO: plot histogram of σ²posterior

[Turing]:  Assume - `σ²` is a parameter
 @~(::ANY, ::ANY) at compiler.jl:76
[Turing]:  Observe - `k` is an observation
 @~(::ANY, ::ANY) at compiler.jl:57


[PG] Sampling... 79%  ETA: 0:00:01

[PG] Finished with
  Running time    = 2.599131478000002;


[PG] Sampling...100% Time: 0:00:03


300-element Array{Float64,1}:
 2.80942
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 1.09958
 ⋮      
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908
 1.06908

## STEP 3: posterior prediction
We'll now write our own function to do posterior prediction given the posterior samples of `σ²`

In [7]:
function posterior_prediction(T::Int64, Δμ_list, σ²samples)
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    k_pred = zeros(n_samples, length(data["Δμ"]))
    k_pred = [rand(Binomial(T, PC(Δμ,σ²))) for Δμ ∈ Δμ_list, σ² ∈ σ²samples]
end

k_pred = posterior_prediction(data["T"], data["Δμ"], σ²posterior)
# so now we have posterior predictive samples in k_pred.
# Each row corresponds to an mcmc sample, each column corresponds to a value of Δμ


# TODO: do summary stats or plotting of posterior predictions, and plot the actual data so we
# can do model critique visually

# TODO: can also do out of sample prediction simply by providing more values for Δμ

10×300 Array{Int64,2}:
  51   51   56   52   50   38   51   54  …   54   50   45   45   56   47   51
  48   53   50   52   55   54   54   57      51   41   59   49   49   49   44
  53   47   46   50   54   56   54   52      56   51   43   55   47   56   47
  55   55   57   55   55   55   61   56      54   66   54   49   52   58   53
  58   56   55   60   54   52   60   47      63   60   55   63   61   59   49
  69   53   68   65   69   68   58   62  …   61   56   63   64   68   62   62
  60   70   71   69   71   75   75   75      71   79   71   86   76   79   70
  84   96   93   97   90   96   90   92      93   94   96   90   95   93   93
  93  100  100  100  100  100  100  100     100  100   99  100  100  100  100
 100  100  100  100  100  100  100  100     100  100  100  100  100  100  100

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001